# [Using EasyOCR with pretrained YOLO for License Plate Recognition](https://youtu.be/9KnARRqbkzY?si=lwrQjDCc9q9ERes-&t=1560)

---

This notebook demonstrates how to use EasyOCR in combination with a pretrained YOLO model to perform license plate recognition. The YOLO model is used to detect license plates in images, and EasyOCR is then used to read the text from the detected plates.


Before running a notebook for the first time, install the required dependencies with:

```bash
	make requirements-nb
```


In [ ]:
from civicpulseML.config import BASE_DIR, CUDA_AVAILABLE, MODELS_DIR, RAW_DIR

## Importing finetuned YOLO model

---


In [2]:
from ultralytics import YOLO
import json

### Reading metadata file

In [3]:
with open(MODELS_DIR / "modeldata.json") as f:
    model_config = json.load(f)

### Getting path of best model

In [4]:
best_path = model_config["yolo"]["weights_path"]

### Loading YOLO model

In [6]:
model = YOLO(BASE_DIR / best_path)

## Initialising EasyOCR Reader

---
